# copynet 

# pointer-generator network

In [1]:
import tensorflow as tf

In [ ]:
class Pointer(tf.keras.layers.Layer):
    
    def __init__(self):
        super(Pointer, self).__init__()
        self.w_s_reduce = tf.keras.layers.Dense(1)
        self.w_i_reduce = tf.keras.layers.Dense(1)
        self.w_c_reduce = tf.keras.layers.Dense(1)
    
    def call(self, context_vector, state, dec_inp):
        return tf.nn.sigmoid(self.w_s_reduce(state)+self.w_c_reduce(context_vector)+self.w_i_reduce(dec_inp))

In [ ]:
class PGN(tf.keras.Model):
  
    def __init__(self, params):
        super(PGN, self).__init__()
        self.encoder = Encoder()
        self.attention = BahdanauAttention()
        self.decoder = Decoder()
        self.pointer = Pointer()
    
    def call_encoder(self, enc_inp):
        enc_hidden = self.encoder.initialize_hidden_state()
        enc_output, enc_hidden = self.encoder(enc_inp, enc_hidden)
        return enc_hidden, enc_output
    
    def call(self, enc_output, dec_hidden, enc_inp, enc_extended_inp,  dec_inp, batch_oov_len):
        predictions = []
        attentions = []
        p_gens = []
        context_vector, _ = self.attention(dec_hidden, enc_output)
        for t in range(dec_inp.shape[1]):
            dec_x, pred, dec_hidden = self.decoder(tf.expand_dims(dec_inp[:, t],1), dec_hidden, enc_output, context_vector)
            context_vector, attn = self.attention(dec_hidden, enc_output)
            p_gen = self.pointer(context_vector, dec_hidden, tf.squeeze(dec_x, axis=1))
      
            predictions.append(pred)
            attentions.append(attn)
            p_gens.append(p_gen)
        # 计算最终的分布
        final_dists = _calc_final_dist()
        return final_dists

# coverage

In [ ]:
def _coverage_loss(attn_dists, padding_mask):
    """
    Calculates the coverage loss from the attention distributions.
    
    """
    coverage = tf.zeros_like(attn_dists[0]) # shape (batch_size, attn_length). Initial coverage is zero.
    covlosses = [] # Coverage loss per decoder timestep. Will be list length max_dec_steps containing shape (batch_size).
    for a in attn_dists:
        covloss = tf.reduce_sum(tf.minimum(a, coverage), [1]) # calculate the coverage loss for this step
        covlosses.append(covloss)
        coverage += a # update the coverage vector
    coverage_loss = _mask_and_avg(covlosses, padding_mask)
    return coverage_loss

In [ ]:
if use_coverage and prev_coverage is not None:
    # self.W_s(values) [batch_sz, max_len, units] self.W_h(hidden_with_time_axis) [batch_sz, 1, units]
    # self.W_c(prev_coverage) [batch_sz, max_len, units]  score [batch_sz, max_len, 1]
    score = self.V(tf.nn.tanh(self.W_s(enc_output) + self.W_h(hidden_with_time_axis) + self.W_c(prev_coverage)))
    # attention_weights shape (batch_size, max_len, 1)
    mask = tf.cast(enc_pad_mask, dtype=score.dtype)
    masked_score = tf.squeeze(score, axis=-1) * mask
    masked_score = tf.expand_dims(masked_score, axis=2)
    attention_weights = tf.nn.softmax(masked_score, axis=1)

    coverage = attention_weights + prev_coverage


In [ ]:
class Pointer(tf.keras.layers.Layer):
    """
    calculate Pgen
    input context_vector [batch_sz,enc_units] dec_hidden [batch_sz,dec_units] dec_inp_context [batch_sz,1,embedding_dim+enc_units]
    output scaler pgen
    """
    def __init__(self):
        super(Pointer, self).__init__()
        self.w_s_reduce = tf.keras.layers.Dense(1)
        self.w_i_reduce = tf.keras.layers.Dense(1)
        self.w_c_reduce = tf.keras.layers.Dense(1)

    def call(self, context_vector, dec_hidden, dec_inp):
        # change dec_inp_context to [batch_sz,embedding_dim+enc_units]
        dec_inp = tf.squeeze(dec_inp, axis=1)
        pgen = tf.nn.sigmoid(self.w_s_reduce(dec_hidden) + self.w_c_reduce(context_vector) + self.w_i_reduce(dec_inp))
        return pgen

# 先验知识其他技巧

In [ ]:
class ScaleShift(Layer):
    """缩放平移变换层（Scale and shift）"""
    def __init__(self, **kwargs):
        super(ScaleShift, self).__init__(**kwargs)
    def build(self, input_shape):
        kernel_shape = (1,) * (len(input_shape)-1) + (input_shape[-1],)
        self.log_scale = self.add_weight(name='log_scale',
                                         shape=kernel_shape,
                                         initializer='zeros')
        self.shift = self.add_weight(name='shift',
                                     shape=kernel_shape,
                                     initializer='zeros')
    def call(self, inputs):
        x_outs = K.exp(self.log_scale) * inputs + self.shift
        return x_outs

- RNN（lstm/gru)参数初始化

- 梯度消减

- Dropout/L2 regularization

- learning rate

- 双向lstm、gru

- 数据预处理

- encoder和decoder的embedding共享参数（也就是用一套词向量）

- 采用预训练词向量

- batch size调小，16，32，64

- hidden states 128，256

- Adam betra1=0.9， betra2=0.999， e=10-8

- learning rate=0.0001， 0.001

- clipping gradient = 2.0

